# https://www.django-rest-framework.org/api-guide/serializers/

In [120]:
import os, django
from rest_framework import generics, viewsets
from django.urls import path
from rest_framework import serializers
from rest_framework.renderers import JSONRenderer # Преобразует объект сериализации в байтовую json строку
from rest_framework.parsers import JSONParser
from io import BytesIO
from rest_framework.response import Response
from paper.models import *
from django.http import QueryDict

In [2]:
# fix notebook
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [3]:
class TestModel:
    '''имитация модели фреймворка django'''
    def __init__(self, title, content):
        self.title = title
        self.content = content

In [4]:
class TestSerializer(serializers.Serializer):
    '''
    сериализатор для TestModel
    * в классе сериализатора должны совпадать атрибуты класса с атрибутами экземпляра класса TestModel *
    '''
    # внутри charfield можно написать условия для сериализатора по которым будет проверяться поле title в json запросe
    title = serializers.CharField() # даем понять сериализатору что в данном атрибуте ожидается строка
    content = serializers.CharField() # на уровне сериализаторов строка и текст представляются одним и тем же классом

In [76]:
# кодирование и преобразование класса TestModel в json формат

# создаем экземпляр класса
model = TestModel("some_title", "some_content")

# далее нам нужно пропустить только что созданный объект через сериализатор TestSerializer
model_sr = TestSerializer(model) # объект сериализатора. Здесь срабатывает метакласс который определен для сериализаторов
# метакласс model_sr создает специальную коллекцию data состоящую из значений локальных атрибутов объекта model
# и представляет его в виде словаря

print(f"Сериализованные данные 'модели' TestModel -> {model_sr.data}") # model_sr.data - сериализованные данные
JSONRenderer().render(model_sr.data) # готовый результат. Модель преобразована в байтовую json строку

Сериализованные данные 'модели' TestModel -> {'title': 'some_title', 'content': 'some_content'}


b'{"title":"some_title","content":"some_content"}'

In [6]:
# Обратное преобразования JSON строки в объект класса TestModel

# имитация поступления запроса от клиента
stream = BytesIO(b' {"title": "hell", "content": "conten"} ')
data = JSONParser().parse(stream)
serializer = TestSerializer(data=data) # получаем объект сериализации

# проверяем корректность принятых данных
serializer.is_valid()

print(serializer.validated_data) # появляется после проверки serializer.is_valid()

OrderedDict([('title', 'hell'), ('content', 'conten')])


In [7]:
# OrderedDict([('title', 'hell'), ('content', 'conten')])
# Теперь с помощью этих данных мы можем создать класс TestModel

In [65]:
# Модель Paper, которая прописана в проекте https://github.com/wizemiller/World-Wide-Blog/blob/main/paper/models.py#L9
# class Paper(models.Model):
#     title = models.CharField(max_length=255)
#     slug = models.SlugField(max_length=255, unique=True, db_index=True)
#     content = models.TextField(blank=True)
#     photo = models.ImageField(upload_to="photos/%Y/%m/%d/", default=None)
#     time_create = models.DateTimeField(auto_now_add=True)
#     time_update = models.DateTimeField(auto_now=True)
#     is_published = models.BooleanField(default=True)
#     cat = models.ForeignKey('Categories', on_delete=models.PROTECT, null=True)

# Сериализатор для модели Paper

class PaperSerializer(serializers.Serializer):
    '''Определение класса сериализатора PaperSerializer, который наследуется от класса Serializer.'''
    
    
    # Все что тут описано, это валидаторы
    
    # read_only - заглушка для валидатора. Данное поле будет в качестве чтения
    
    # Определение поля title как CharField с максимальной длиной 255 символов.
    title = serializers.CharField(max_length=255)

    # Определение поля slug как SlugField с максимальной длиной 255 символов и разрешением пустого значения.
    slug = serializers.SlugField(max_length=255, allow_blank=True)
    
    # Определение поля content как CharField с возможностью отсутствия значения.
    content = serializers.CharField(required=False)
    
    # Определение поля photo как ImageField с возможностью отсутствия значения.
    photo = serializers.ImageField(required=False, read_only=True)
    
    # Определение поля time_create как DateTimeField только для чтения.
    time_create = serializers.DateTimeField(read_only=True)
    
    # Определение поля time_update как DateTimeField только для чтения.
    time_update = serializers.DateTimeField(read_only=True)
    
    # Определение поля is_published как BooleanField со значением по умолчанию True.
    is_published = serializers.BooleanField(default=True)
    
    # Определение поля cat как PrimaryKeyRelatedField с возможностью отсутствия значения и связью с моделью Categories.
    cat = serializers.PrimaryKeyRelatedField(queryset=Categories.objects.all(), required=False)

    
    # Методы create и update будут выбираться по такому принципу
    # PaperSerializer(data=request.data, instance=Model_object like Paper.objects.get(pk=pk)) --> update
    # PaperSerializer(data=request.data) --> create
    
    def create(self, validated_data): # для добавления записи в бд вызывается через метод save()
        '''Когда мы вызываем is_validate у нас генерируется словарь validated_data'''
        return Paper.objects.create(**validated_data)

    def update(self, instance, validated_data):
        # instance - ссылка на объект Paper
        # validated_data - словарь из проверенных данных которые нужно изменить в базе данных
        instance.title = validated_data.get('title', instance.title)
        instance.slug = validated_data.get('slug', instance.slug)
        instance.content = validated_data.get('content', instance.content)
        instance.photo = validated_data.get('photo', instance.photo)
        instance.is_published = validated_data.get('is_published', instance.is_published)
        instance.cat = validated_data.get('cat', instance.cat)
        instance.save()
        return instance

In [9]:
# Возвращаем данные клиенту

items = Paper.objects.all() # формируется список объектов класса Paper
# Поскольку тут используется список записей мы добавим many=True
Response({"posts": PaperSerializer(items, many=True).data}).data
# response преобразовывает все в байтовую json строку по сути тоже самое что и JSONRenderer
# Здесь все происходит тоже самое что и когда мы делали "кодирование и преобразование класса TestModel в json формат"

{'posts': [OrderedDict([('title', 'stateman2'), ('slug', 'hellman'), ('content', 'kasjeklfjaseklgjlasjfkljas'), ('photo', '/media/photos/2023/08/12/Screenshot_14.png'), ('time_create', '2023-08-12T10:12:43.140047+03:00'), ('time_update', '2023-08-12T10:12:43.140047+03:00'), ('is_published', True), ('cat', 1)]), OrderedDict([('title', 'stateman'), ('slug', 'slugger'), ('content', 'ausehjgkalsejgiou2h3giolj2k3lgejsalkjg;askjefasef'), ('photo', '/media/photos/2023/08/12/Screenshot_4.png'), ('time_create', '2023-08-12T10:12:01.045301+03:00'), ('time_update', '2023-08-12T10:12:01.045301+03:00'), ('is_published', True), ('cat', 1)])]}

In [75]:
# Клиент отправляет данные через post запрос и хочет добавить статью на сайте

# curl -X POST --data 'title=apistate&slug=apistateslug&content=helloworld&cat=1' http://127.0.0.1:8000/api/v1/paper

request = QueryDict(
    'title=apistate&slug=apistateslug&content=helloworld&cat=1'
)

serializer = PaperSerializer(data=request)
serializer.is_valid(raise_exception=True) # raise_excepiton для клиента, если он не направил все нужные данные в теле запроса

# Paper.objects.create(title=request['title'].....) # после валидации данных мы можем добавлять запись в БД

True

Изначально мы сделали сериализатор на базе класса serializers.Serializer это довольно частая схема, поскольку он позволяет
получать данные в json формате, изменять уже существующие записи в таблице базы данных
для этого было использована таблица Paper и ORM Django
сериализатор напрямую связан с таблицей базы данных это распространенная схема, поэтому для таких целей существует класс
сериализатора serializers.ModelSerializer и значительно упрощает работу

In [77]:
# serializers.py
class PaperModelSerializer(serializers.ModelSerializer):
    class Meta:
        model = Paper
        fields = "__all__" # можно выводить определенные поля через ('title', 'content', .....) мы же укажем все сразу

# Поскольку, были разработаны упрощения для сериализаторов моделей посредством класса ModelSerializer, то и для 
# Классов-представлений тоже есть свои классы для простоты
# https://www.django-rest-framework.org/api-guide/serializers/

In [115]:
# views.py

from rest_framework import generics

class PaperAPIview(generics.ListCreateAPIView): # чтение списка данных по get запросу и создание по POST
    queryset = Paper.objects.all() # данные, которые будут возвращаться
    serializer_class = PaperSerializer # класс сериализации который обрабатывает queryset

    
class PaperAPIview(generics.ListAPIView): # чтение списка данных по get запросу
    queryset = Paper.objects.all() # данные, которые будут возвращаться
    serializer_class = PaperSerializer # класс сериализации который обрабатывает queryset
    
    
class PaperUpdateAPIView(generics.UpdateAPIView): # изменение записи по put, patch запросу
    ''' tests -> curl -X PUT -d title=hello -d slug=world http://127.0.0.1:8000/api/v1/paper/7/ '''
    queryset = Paper.objects.all() # ленивый запрос
    serializer_class = PaperSerializer # класс сериализации который обрабатывает queryset
    
# CRUD - generics.RetrieveUpdateDestroyAPIView

Также есть viewset чтобы каждый раз не дублировать queryset и serializer_class в классе-представлений
[ViewSets](https://www.django-rest-framework.org/api-guide/viewsets/)

In [128]:
# Мы будем использовать ModelViewSet поскольку наше представление работает с моделями

# views.py

class PaperViewSet(viewsets.ModelViewSet):
    queryset = Paper.objects.all()
    serializer_class = PaperSerializer

# urls.py
'''
list - возвращает список
create - создает новую запись
retrieve - выделяет запись
update - меняет запись
destroy - удаляет запись
'''

# вот таких маршрутов может быть множество, поэтому этот процесс автоматизирован на уровне DRF с использованием роутеров
urlpatterns = [
#     .....
    path('api/v1/papers/<int:pk>/', PaperViewSet.as_view({'get': 'retrieve'}))
#     ....
]


# class ModelViewSet(mixins.CreateModelMixin,
#                    mixins.RetrieveModelMixin,
#                    mixins.UpdateModelMixin,
#                    mixins.DestroyModelMixin,
#                    mixins.ListModelMixin,
#                    GenericViewSet):
#     """
#     A viewset that provides default `create()`, `retrieve()`, `update()`,
#     `partial_update()`, `destroy()` and `list()` actions.
#     """
#     pass

# - CreateModelMixin - реализует метод create(), который создает новый объект модели.
# - RetrieveModelMixin - реализует метод retrieve(), который получает объект модели по его идентификатору.
# - UpdateModelMixin - реализует метод update(), который обновляет объект модели целиком.
# - DestroyModelMixin - реализует метод destroy(), который удаляет объект модели.
# - ListModelMixin - реализует метод list(), который возвращает список всех объектов модели.

# GenericViewSet - это базовый класс для всех ViewSet'ов в Django REST Framework, который 
# предоставляет общую функциональность и не определяет никаких дополнительных методов.